In [3]:
import os 
import pandas as pd
import numpy as np
import unicodedata
import pyodbc
from datetime import date 

In [5]:
ruta_base = os.getcwd()
ruta_csv = os.path.join(ruta_base,'Prueba Técnica', 'Data', 'netflix_titles.csv')
df= pd.read_csv(ruta_csv)

In [7]:
df.drop_duplicates(inplace=True)

In [9]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [11]:
df['director'] = df['director'].fillna("No Precisa")
df['cast'] = df['cast'].fillna("No Precisa")
df['country'] = df['country'].fillna("No Precisa")
df['rating'] = df['rating'].fillna("No Precisa")
df['duration'] = df['duration'].fillna("No Precisa")

In [13]:
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

In [15]:
df['cast'] = df['cast'].apply(lambda x: [i.strip() for i in x.split(',')] if x else [])
df['listed_in'] = df['listed_in'].apply(lambda x: [i.strip() for i in x.split(',')] if x else [])
df['country'] = df['country'].apply(lambda x: [i.strip() for i in x.split(',')] if x else [])

In [17]:
def normalizar_texto(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
    return text.title()

In [19]:
def normalizar_cast(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
    return text.title()

In [21]:
df['cast'] = df['cast'].apply(
    lambda x: ', '.join([normalizar_texto(i) for i in x]) if isinstance(x, list) else normalizar_cast(x))
df['listed_in'] = df['listed_in'].apply(
    lambda x: ', '.join([normalizar_texto(i) for i in x]) if isinstance(x, list) else normalizar_cast(x))
df['country'] = df['country'].apply(
    lambda x: ', '.join([normalizar_texto(i) for i in x]) if isinstance(x, list) else normalizar_cast(x))
df['director'] = df['director'].apply(
    lambda x: ', '.join([normalizar_texto(i) for i in x]) if isinstance(x, list) else normalizar_cast(x))

In [23]:
def get_or_create_id(table, column, value):
    if pd.isna(value) or str(value).strip() == "":
        return None
    try:
        cursor.execute(f"SELECT id FROM {table} WHERE {column} = ?", (value,))
        row = cursor.fetchone()
        if row:
            return row[0]

        cursor.execute(f"INSERT INTO {table} ({column}) VALUES (?)", (value,))
        cursor.execute("SELECT SCOPE_IDENTITY()")
        new_id = cursor.fetchone()[0]
        conn.commit()
        return new_id
    except Exception as e:
        print(f"Error insertando en {table}: {value} -> {e}")
        return None

In [43]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-E4JCHGB;DATABASE=PRUEBA_DELFOSTI;Trusted_Connection=yes;')
cursor = conn.cursor()

In [47]:
def normalize_text(text):
    if pd.isna(text):
        return None
    s = str(text).strip()
    if not s or s.lower() == "no precisa":
        return None
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("utf-8")
    s = s.title()
    letters = [ch for ch in s if ch.isalpha()]
    if len(letters) < 2:
        return None
    return s

In [ ]:
def normalize_actor(text):
    if pd.isna(text):
        return None
    s = str(text).strip()
    if not s or s.lower() == "no precisa":
        return None
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("utf-8")
    if sum(ch.isalpha() for ch in s) < 1 or len(s) < 2:
        return None
    return s

In [ ]:
def safe_date(value):
    if pd.isna(value):
        return None
    try:
        dt = pd.to_datetime(value).date()
        if dt < date(1753,1,1) or dt > pd.Timestamp.today().date():
            return None
        return dt.isoformat()
    except:
        return None

In [ ]:

def get_or_create_id(table, column, value):
    val = normalize_text(value)
    if not val:
        return None
    cursor.execute(f"SELECT Id FROM {table} WHERE {column} = ?", (val,))
    row = cursor.fetchone()
    if row:
        return row[0]
    cursor.execute(f"INSERT INTO {table} ({column}) VALUES (?)", (val,))
    cursor.execute("SELECT SCOPE_IDENTITY()")
    new = cursor.fetchone()
    conn.commit()
    return new[0] if new else None

In [ ]:
def insert_pelicula(row):
    try:
        # Preparar valores
        id_show = row.get("show_id")
        title = normalize_text(row.get("title"))
        tipo = normalize_text(row.get("type"))
        duracion = normalize_text(row.get("duration"))
        anio = int(row["release_year"]) if pd.notna(row["release_year"]) else None
        desc = row.get("description")
        fecha = safe_date(row.get("date_added"))

        dir_id = get_or_create_id("Directores", "Nombre", row.get("director"))
        rate_id = get_or_create_id("Ratings",   "Nombre", row.get("rating"))

      
        cursor.execute("""
            INSERT INTO Pelicula (
                Id_Show, Titulo, Tipo, Duracion, Anio_Estreno, Descripcion,
                Id_Director, Id_Rating, Fecha
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (id_show, title, tipo, duracion, anio, desc, dir_id, rate_id, fecha))
        cursor.execute("SELECT SCOPE_IDENTITY()")
        new = cursor.fetchone()
        pelicula_id = new[0] if new else None
        conn.commit()

      
        cast_list = row.get("cast_list", [])
        for actor in cast_list:
            act_id = get_or_create_id("Actores", "Nombre", actor, normalizer=normalize_actor)
            if act_id and pelicula_id:
                cursor.execute(
                "INSERT INTO Actores_Pelicula (Id_Titulo, Id_Actor) VALUES (?, ?)",
                (pelicula_id, act_id)
            )
       
        for pais in row.get("country_list", []):
            pais_id = get_or_create_id("Paises", "Nombre", pais)
            if pelicula_id and pais_id:
                cursor.execute(
                    "INSERT INTO Paises_Pelicula (Id_Titulo, Id_Pais) VALUES (?, ?)",
                    (pelicula_id, pais_id)
                )
      
        for genero in row.get("genres_list", []):
            gen_id = get_or_create_id("Generos", "Nombre", genero)
            if pelicula_id and gen_id:
                cursor.execute(
                    "INSERT INTO Genero_Pelicula (Id_Titulo, Id_Genero) VALUES (?, ?)",
                    (pelicula_id, gen_id)
                )

        conn.commit()
    except Exception as e:
        print(f"Error insertando '{row.get('title')}': {e}")
        conn.rollback()

In [49]:
if 'cast_list' not in df.columns:
    df['cast_list'] = df['cast'].fillna("").apply(lambda x: [i.strip() for i in x.split(',')] if x else [])
if 'country_list' not in df.columns:
    df['country_list'] = df['country'].fillna("").apply(lambda x: [i.strip() for i in x.split(',')] if x else [])
if 'genres_list' not in df.columns:
    df['genres_list'] = df['listed_in'].fillna("").apply(lambda x: [i.strip() for i in x.split(',')] if x else [])

df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df = df[df['date_added'].notna() & 
        df['date_added'].between(pd.Timestamp('1753-01-01'), pd.Timestamp.today())].copy()

In [ ]:
for _, row in df.iterrows():
    insert_pelicula(row)

cursor.close()
conn.close()